<a href="https://www.kaggle.com/code/tusharaggarwal27/bank-term-deposit-analysis?scriptVersionId=122655997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


<div style="color:white;
           display:fill;
           border-radius:20px;
           background-color:#F5DEB3;
           font-size:170%;
           font-family:Helvetica, Sans-Serif;
           letter-spacing:0.5px">
<p style="padding: 10px;
          text-align: center;
          font-size:150%;
          color:blue;">
          💵Bank term deposit analysis💵
            
</p>
<style>
        h1{text-align: center;}
</style>  
    
</div>

<p style="font-family:cursive; font-size:15px; color:'darkcyan';">I brewed this notebook from scratch, If this notebook helped, please consider upvoting and cite me if sharing ,Thank you!</p>
<p style="width: 250px;padding: 15px;background: papayawhip;border-radius:10px;font-size:15px;border-radius:20px">
    <a href="https://www.linkedin.com/in/tusharaggarwalinseec/" target="_blank">Lets connect on LinkedIn!</a>
    
   </p>
<p style="width: 250px;padding: 15px;background: papayawhip;border-radius:10px;font-size:15px;border-radius:20px">
<a href="https://github.com/tushar2704" target="_blank">Follow me on Github too!</a> </p>
<p style="width:270px;padding: 15px;background: papayawhip;border-radius:10px;font-size:15px;border-radius:20px">
    <a href="https://medium.com/@tushar_aggarwal" target="_blank">Also checkout my Medium posts!</a>
    
</p>


<div class="alert alert-info" role="alert" style="font-size:17px;border-radius:20px">
    <b>In this Project, I am building a model that predict term deposit application by individuals based on Bank term deposit dataset with H2O </b>
  </div>

<p style="width: 1000px;padding: 20px;background: papayawhip;border-radius:10px;font-size:25px;border-radius:20px">
    Importing the required Packages
</p>

In [1]:
#Data manipulation imports
import numpy as np
import pandas as pd
pd.options.display.max_rows=999
#Data Visualization imports
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#Modeling imports
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score,mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import h2o

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam
#Additional imports
from math import sqrt

<p style="width: 1000px;padding: 20px;background: papayawhip;border-radius:10px;font-size:25px;border-radius:20px">
    Loading and checking data
</p>

In [2]:
xlsx =pd.ExcelFile("/kaggle/input/bank-term-deposit-marketing-analysis/bank_term_deposit_marketing_analysis.xlsx")

In [3]:
xlsx.sheet_names

['PROCEDURE',
 'DATA DESCRIPTION',
 'Step 1 - Collect Information',
 'CLIENT_INFO',
 'LOAN_HISTORY',
 'MARKETING HISTORY',
 'SUBSCRIPTION HISTORY',
 'Step 2 - Merge Information',
 'CLIENT_MERGE',
 'Step 3 - Marketing Analysis',
 'DAILY RANGE',
 'JOB ANALYSIS',
 'Sheet3']

In [4]:
client_info = pd.read_excel(xlsx, "CLIENT_INFO")
loan_info = pd.read_excel(xlsx, "LOAN_HISTORY")
marketing_info = pd.read_excel(xlsx, "MARKETING HISTORY")
subscription_info = pd.read_excel(xlsx, "SUBSCRIPTION HISTORY")

In [5]:
client_info.head(5)

,ID,AGE,JOB,MARITAL,EDUCATION
0,2836,58.0,management,married,tertiary
1,2837,44.0,technician,single,secondary
2,2838,33.0,entrepreneur,married,secondary
3,2839,47.0,blue-collar,married,unknown
4,2840,33.0,unknown,single,unknown


In [6]:
loan_info.head(5)

,ID,DEFAULT,BALANCE,HOUSING,LOAN
0,2836,no,2143.0,yes,no
1,2837,no,29.0,yes,no
2,2838,no,2.0,yes,yes
3,2839,no,1506.0,yes,no
4,2840,no,1.0,no,no


In [7]:
marketing_info.head(5)

,ID,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME
0,2836,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown
1,2837,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown
2,2838,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown
3,2839,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown
4,2840,unknown,5.0,may,198.0,1.0,-1.0,0.0,unknown


In [8]:
subscription_info.head(5)

,ID,TERM_DEPOSIT
0,2836,no
1,2837,no
2,2838,no
3,2839,no
4,2840,no


In [9]:
#Merging on 'id'
df = pd.merge(loan_info,client_info, on=['ID'])
df =pd.merge(df, marketing_info,on=['ID'] )
df =pd.merge(df, subscription_info, on=['ID'])
df.head(5)

,ID,DEFAULT,BALANCE,HOUSING,LOAN,AGE,JOB,MARITAL,EDUCATION,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME,TERM_DEPOSIT
0,2836,no,2143.0,yes,no,58.0,management,married,tertiary,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,no
1,2837,no,29.0,yes,no,44.0,technician,single,secondary,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,2838,no,2.0,yes,yes,33.0,entrepreneur,married,secondary,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no
3,2839,no,1506.0,yes,no,47.0,blue-collar,married,unknown,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown,no
4,2840,no,1.0,no,no,33.0,unknown,single,unknown,unknown,5.0,may,198.0,1.0,-1.0,0.0,unknown,no


In [10]:
#Droppinf "ID" column
df =df.drop(columns='ID')


<p style="width: 1000px;padding: 20px;background: papayawhip;border-radius:10px;font-size:25px;border-radius:20px">
    Data prepration for H2O
</p>

In [11]:
#Initalzing h2o cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu220.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu220.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpkbyy25n2
  JVM stdout: /tmp/tmpkbyy25n2/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpkbyy25n2/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_unknownUser_j9ofqg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [12]:
#Converting DataFrame to H2O frame
h2o_df= h2o.H2OFrame(df)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [13]:
#Describing H2O Frame
h2o_df.describe()

Rows:45211
Cols:17

,DEFAULT,BALANCE,HOUSING,LOAN,AGE,JOB,MARITAL,EDUCATION,CONTACT,DAY,MONTH,DURATION,CAMPAIGN,PDAYS,PREVIOUS,POUTCOME,TERM_DEPOSIT
type,enum,int,enum,enum,int,enum,enum,enum,enum,int,enum,int,int,int,int,enum,enum
mins,,-8019.0,,,18.0,,,,,1.0,,0.0,1.0,-1.0,0.0,,
mean,,1362.2720576850802,,,40.936210214328106,,,,,15.80641879188694,,258.1630797814691,2.763840658246891,40.197827962221574,0.5803233726305549,,
maxs,,102127.0,,,95.0,,,,,31.0,,4918.0,63.0,871.0,275.0,,
sigma,,3044.7658291685234,,,10.618762040975398,,,,,8.32247615304459,,257.52781226517124,3.0980208832791747,100.12874599059825,2.3034410449312213,,
zeros,,3514,,,0,,,,,0,,3,0,0,36954,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,no,2143.0,yes,no,58.0,management,married,tertiary,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,no
1,no,29.0,yes,no,44.0,technician,single,secondary,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,no,2.0,yes,yes,33.0,entrepreneur,married,secondary,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no


In [14]:
#Defining predictotors and response variables
train, test =h2o_df.split_frame(ratios=[0.75])
x=train.columns
y='TERM_DEPOSIT'
x.remove(y)

<p style="width: 1000px;padding: 20px;background: papayawhip;border-radius:10px;font-size:25px;border-radius:20px">
    H2O
</p>

<p style="width: 1000px;padding: 20px;background: papayawhip;border-radius:10px;font-size:25px;border-radius:20px">
    ANN with dropout works best out of above! App coming soon!
</p>